Target dataframe

filename | repository | workflow name | commit datetime | commit_id | action used | action version | vulnerable versions | fixing patch | Is Vulnerable | vulnerability disclosure

In [8]:
!pip install pyyaml
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 33.6 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 34.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [20]:
import yaml
import re

def extract_github_actions(yml_path):
    with open(yml_path, "r") as file:
        data = yaml.safe_load(file)

    actions = []

    def search_uses(obj):
        """Recursively search for 'uses' keys in the workflow YAML structure."""
        if isinstance(obj, dict):
            for key, value in obj.items():
                if key == "uses" and isinstance(value, str):
                    # Match action and version, e.g. "actions/checkout@v3"
                    match = re.match(r"([^@]+)@(.+)", value)
                    if match:
                        actions.append({
                            "action": match.group(1),
                            "version": match.group(2)
                        })
                    else:
                        actions.append({"action": value, "version": "N/A"})
                else:
                    search_uses(value)
        elif isinstance(obj, list):
            for item in obj:
                search_uses(item)

    search_uses(data)
    return actions


if __name__ == "__main__":
    yml_path = "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml"  # e.g. ".github/workflows/ci.yml"
    actions = extract_github_actions(yml_path)

    print("🧩 GitHub Actions used in workflow:\n")
    print(len(actions), "actions found:\n")
    for act in actions:
        print(f"- {act['action']} (version: {act['version']})")

ParserError: while parsing a block mapping
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 10, column 5
expected <block end>, but found '-'
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 13, column 5

In [6]:
import re
from pathlib import Path

def parse_workflow_filename(full_path):
    path = Path(full_path)
    filename = path.name
    repository = str(path.parent)  # repository is the folder path

    # Regex to extract workflow name, datetime, and commit id
    pattern = r"^(?P<workflow>.+)__" \
              r"(?P<datetime>\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2}Z)__" \
              r"(?P<commit>[a-f0-9]{40})\.yml$"

    match = re.match(pattern, filename)
    if match:
        workflow_name = match.group("workflow")
        commit_datetime = match.group("datetime").replace("-", ":", 2).replace("-", "", 2)  # adjust hour-min-sec
        commit_datetime = commit_datetime.replace("-", ":", 2).replace("-", ":", 1)  # correct formatting to T13:53:55Z
        commit_id = match.group("commit")
        return {
            "filename": filename,
            "repository": repository,
            "workflow_name": workflow_name,
            "commit_datetime": commit_datetime,
            "commit_id": commit_id
        }
    else:
        return None

# Example usage
full_path = "yuri-moraes/sistema_erp/neon-preview-branches-for-pull-requests-gentle-field-50680377__2024-10-07T13-53-55Z__de100a5bff3f15e623b947464d458e0d641d774a.yml"

parsed = parse_workflow_filename(full_path)
print(parsed)

{'filename': 'neon-preview-branches-for-pull-requests-gentle-field-50680377__2024-10-07T13-53-55Z__de100a5bff3f15e623b947464d458e0d641d774a.yml', 'repository': 'yuri-moraes/sistema_erp', 'workflow_name': 'neon-preview-branches-for-pull-requests-gentle-field-50680377', 'commit_datetime': '2024:10:07T135355Z', 'commit_id': 'de100a5bff3f15e623b947464d458e0d641d774a'}


In [22]:
import os
import pandas as pd

# Assuming these two functions are already defined:
# 1. extract_github_actions(yml_path) -> returns list of {'action': ..., 'version': ...}
# 2. parse_workflow_filename(full_path) -> returns dict with filename, repository, workflow_name, commit_datetime, commit_id

def collect_workflow_data(root_folder):
    data = []
    num_correct_files = 0
    num_error_files = 0

    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.endswith(".yml") or file.endswith(".yaml"):
                full_path = os.path.join(dirpath, file)
                
                # Parse workflow filename info
                workflow_info = parse_workflow_filename(full_path)
                if workflow_info is None:
                    continue  # skip files that don't match the expected pattern

                # Extract GitHub actions from the YML
                # Try extracting GitHub actions safely
                try:
                    actions = extract_github_actions(full_path)
                except Exception as e:
                    num_error_files += 1
                    print(f"⚠️ Error processing file {full_path}: {e}")
                    data.append({
                        "filename": workflow_info["filename"],
                        "repository": workflow_info["repository"],
                        "workflow_name": workflow_info["workflow_name"],
                        "commit_datetime": workflow_info["commit_datetime"],
                        "commit_id": workflow_info["commit_id"],
                        "action_used": "ERROR",
                        "action_version": "ERROR",
                        "vulnerable_versions": None,
                        "fixing_patch": None,
                        "is_vulnerable": None
                    })
                    continue  # skip to the next file

                num_correct_files += 1

                # Combine each action with workflow info
                for act in actions:
                    data.append({
                        "filename": workflow_info["filename"],
                        "repository": workflow_info["repository"],
                        "workflow_name": workflow_info["workflow_name"],
                        "commit_datetime": workflow_info["commit_datetime"],
                        "commit_id": workflow_info["commit_id"],
                        "action_used": act.get("action"),
                        "action_version": act.get("version"),
                        "vulnerable_versions": None,
                        "fixing_patch": None,
                        "is_vulnerable": None
                    })

    print(f"✅ Processed {num_correct_files} files successfully.")
    print(f"⚠️ Encountered errors in {num_error_files} files.")

    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

# Usage
root_folder = "dataset/workflows_from_dependents"
df = collect_workflow_data(root_folder)

# Preview
print(df.head())


⚠️ Error processing file dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml: while parsing a block mapping
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 10, column 5
expected <block end>, but found '-'
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 13, column 5
⚠️ Error processing file dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-50-54Z__567d7d276bf6755226a98649e3cb8f34b3953a83.yml: while parsing a block mapping
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-50-54Z__567d7d276bf6755226a98649e3cb8f34b3953a83.yml", line 10, column 5
expected <block end>, but found '-'
  i

KeyboardInterrupt: 